In [1]:
import pandas as pd

In [2]:
stocks = pd.read_csv("stocks-df.csv")
industries = pd.read_csv("industry-data.csv")

In [7]:
print(f"Containing Stock Data from {stocks['Date'].min()} to {stocks['Date'].max()}")

Containing Stock Data from 1990-01-02 to 2022-01-11


In [11]:
stocks = stocks.merge(industries, how="left", left_on="Stock", right_on="symbol")
stocks

,Date,Stock,Open,High,Low,Close,Volume,Change Pct,Unnamed: 0,symbol,industry
0,2013-01-02,20MICRONS,80.35,80.55,77.50,77.80,1088880,-3.23,0.0,20MICRONS,Mining/Minerals
1,2013-01-03,20MICRONS,78.58,79.78,76.00,76.40,590180,-1.80,0.0,20MICRONS,Mining/Minerals
2,2013-01-04,20MICRONS,76.50,77.25,75.50,75.83,380850,-0.75,0.0,20MICRONS,Mining/Minerals
3,2013-01-07,20MICRONS,76.88,78.00,74.00,74.90,1144204,-1.23,0.0,20MICRONS,Mining/Minerals
4,2013-01-08,20MICRONS,74.97,75.97,74.25,75.28,605020,0.51,0.0,20MICRONS,Mining/Minerals
...,...,...,...,...,...,...,...,...,...,...,...
5440147,2022-01-07,ZYDUSWELL,1866.00,1879.00,1840.00,1844.65,27888,-1.35,2022.0,ZYDUSWELL,Pharmaceuticals
5440148,2022-01-10,ZYDUSWELL,1850.25,1865.00,1840.10,1859.15,11310,0.79,2009.0,ZYDUSWELL,Pharmaceuticals
5440149,2022-01-10,ZYDUSWELL,1850.25,1865.00,1840.10,1859.15,11310,0.79,2022.0,ZYDUSWELL,Pharmaceuticals
5440150,2022-01-11,ZYDUSWELL,1865.00,1868.60,1848.00,1853.15,6106,-0.32,2009.0,ZYDUSWELL,Pharmaceuticals


In [12]:
stocks.to_csv("stocks-with-industry.csv")

In [2]:
import pandas as pd
stocks = pd.read_csv("stocks-with-industry.csv")

In [6]:
stocks = stocks.set_index("Date").drop(columns="symbol")

In [12]:
from tqdm.auto import tqdm
company_columns_dict = {}

for company_symbol in tqdm(stocks.Stock.unique()):
    company_columns_dict[company_symbol] = stocks[stocks.Stock == "20MICRONS"]["Open"]

  0%|          | 0/1055 [00:00<?, ?it/s]

In [16]:
stocks = pd.DataFrame(company_columns_dict)

In [17]:
stocks.to_csv("company-datewise.csv")

In [1]:
import pandas as pd
pd.read_csv("/Users/irsh/Documents/workdir/nse2vec/data/embedding_data_files/Data/historical_stocks.csv").sector.unique()

array(['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES',
       'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE',
       'CONSUMER DURABLES', nan, 'ENERGY', 'MISCELLANEOUS', 'SECTOR',
       'TRANSPORTATION', 'CONSUMER NON-DURABLES'], dtype=object)

In [2]:
industry_df = pd.read_csv("/Users/irsh/Documents/workdir/nse2vec/data/industry-data.csv")
len(set([z[0] for z in industry_df.industry.str.split(" ")]))

112

In [3]:
industry_df

,Unnamed: 0,symbol,industry
0,0,20MICRONS,Mining/Minerals
1,1,360ONE,Finance & Investments
2,2,3MINDIA,Diversified
3,3,3PLAND,Finance & Investments
4,4,3IINFOLTD,IT Consulting & Software
...,...,...,...
2021,2021,ZYDUSLIFE,Pharmaceuticals
2022,2022,ZYDUSWELL,Pharmaceuticals
2023,2023,ECLERX,IT Enabled Services
2024,2024,EMUDHRA,IT Enabled Services


In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
chat = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    groq_api_key="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"
)

In [11]:
from langchain_core import pydantic_v1
class Sector(pydantic_v1.BaseModel):
    sector: str = pydantic_v1.Field(description="One category amongst ['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES', 'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE','CONSUMER DURABLES', 'ENERGY','TRANSPORTATION', 'CONSUMER NON-DURABLES']")

system = """\
You are a bot only allowed to output one of ['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES',
       'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE',
       'CONSUMER DURABLES', 'ENERGY',
       'TRANSPORTATION', 'CONSUMER NON-DURABLES'] whichever seems the closest to the category in the input\
"""

prompt_template = ChatPromptTemplate.from_messages([("system", system), ("human", "Tell me which is the most similar category amongst ['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES', 'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE','CONSUMER DURABLES', 'ENERGY','TRANSPORTATION', 'CONSUMER NON-DURABLES'] to this one: {text}")])

chain = prompt_template | chat.with_structured_output(Sector)


In [27]:
from tqdm.auto import tqdm
import time

industry_sector_map = {'Mining/Minerals': 'BASIC INDUSTRIES',
 'Finance & Investments': 'FINANCE',
 'Diversified': 'CONSUMER DURABLES',
 'IT Consulting & Software': 'TECHNOLOGY',
 'Engineering - General': 'CAPITAL GOODS',
 'Electric Equipment - General': 'CAPITAL GOODS',
 'Trading & Distribution': 'CONSUMER DURABLES',
 'Cement': 'BASIC INDUSTRIES',
 'Food Processing - Bakery/Dairy/Fruits/Others': 'CONSUMER NON-DURABLES',
 'Packaging & Containers': 'CONSUMER DURABLES',
 'Financial Technologies (Fintech)': 'FINANCE',
 'Castings/Foundry': 'BASIC INDUSTRIES',
 'Realty': 'FINANCE',
 'Steel - Tubes/Pipes': 'BASIC INDUSTRIES',
 'Construction & Engineering': 'CAPITAL GOODS',
 'Auto Ancl - Others': 'CONSUMER DURABLES',
 'Logistics - Warehousing/Supply Chain/Others': 'TRANSPORTATION',
 'Agricultural Products': 'CONSUMER NON-DURABLES',
 'IT Enabled Services': 'TECHNOLOGY',
 'Textiles - Processing/Texturising': 'CONSUMER DURABLES',
 'Finance - Housing': 'FINANCE',
 'Oil Drilling And Exploration': 'ENERGY'}

for ind in tqdm(industry_df.industry.unique()[22:]):
    print(f"Processing {ind}", end="\t\t\t\t\r")
    industry_sector_map[ind] = chain.invoke(ind).sector
    # time.sleep(1)

  0%|          | 0/162 [00:00<?, ?it/s]

In [28]:
len(industry_sector_map)

184

In [31]:
def industry_to_sector(x):
    return industry_sector_map[x]


In [ ]:
industry_sector_map

In [34]:
industry_df['sector'] = industry_df['industry'].apply(industry_to_sector)

In [42]:
pd.DataFrame({"industry": industry_sector_map.keys(), "sector": industry_sector_map.values()}).to_csv("industry-sector-map.csv")

In [44]:
industry_df.drop(columns="Unnamed: 0").to_csv("industry-data.csv")

In [55]:
import pandas as pd
pd.read_csv("embedding_data_files/Data/historical_stocks.csv")

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS
...,...,...,...,...,...
6455,ZOES,NYSE,"ZOE&#39;S KITCHEN, INC.",CONSUMER SERVICES,RESTAURANTS
6456,ZTS,NYSE,ZOETIS INC.,HEALTH CARE,MAJOR PHARMACEUTICALS
6457,ZTO,NYSE,ZTO EXPRESS (CAYMAN) INC.,TRANSPORTATION,TRUCKING FREIGHT/COURIER SERVICES
6458,ZUO,NYSE,"ZUORA, INC.",TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE


In [56]:
pd.read_csv("embedding_data_files/Data/returns_df_611.csv")

,Unnamed: 0,AAPL,AB,ABM,ABT,ABX,ACXM,ADBE,ADI,ADM,...,WTS,WWW,WY,XCRA,XLNX,XOM,XRAY,XRX,ZION,ZIXI
0,2000-01-03,0.088754,-0.006263,-0.003067,-0.036145,0.000000,0.041667,-0.025093,-0.030242,-0.010309,...,-0.029661,-0.057143,-0.026980,-0.050279,0.028180,-0.027929,-0.005291,0.066116,-0.062302,-0.061514
1,2000-01-04,-0.084310,-0.004202,0.003077,-0.028571,-0.014134,-0.005000,-0.083889,-0.050589,-0.010417,...,0.000000,0.000000,-0.037567,-0.052941,-0.022727,-0.019154,0.000000,-0.046512,-0.048423,-0.018487
2,2000-01-05,0.014634,0.004219,-0.012270,-0.001838,0.017921,0.010050,0.019771,0.014599,-0.015789,...,-0.008734,-0.024242,0.052974,0.043478,-0.021888,0.054516,0.017285,0.051490,-0.001183,0.021403
3,2000-01-06,-0.086538,0.016807,0.003106,0.034991,0.000000,0.009950,0.008163,-0.027338,0.005348,...,0.008811,0.000000,0.046778,-0.062500,-0.086713,0.051698,0.001309,-0.020619,0.014218,-0.054483
4,2000-01-07,0.047368,0.033058,0.003096,0.010676,-0.024648,0.004926,0.048583,0.028107,0.015957,...,-0.021834,0.024845,-0.032884,0.047619,0.113323,-0.002935,-0.002611,0.023684,0.002336,0.078014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4687,2018-08-20,-0.009744,0.006723,0.013815,-0.001236,0.011976,0.005188,0.016524,-0.008881,0.004352,...,0.001804,0.001046,-0.004488,0.000700,0.001408,0.007028,0.004913,0.011507,0.004653,-0.012590
4688,2018-08-21,-0.001949,0.001669,0.020440,0.001702,0.003945,0.012118,0.006967,0.023256,-0.000591,...,0.014406,0.005486,-0.006763,0.004199,0.018279,0.000254,0.002831,0.020550,0.004446,0.009108
4689,2018-08-22,0.000047,0.001667,0.000910,0.014208,0.019646,0.008426,0.016064,0.020851,-0.004730,...,-0.011834,0.001299,0.001418,0.003484,0.007595,0.014335,-0.001796,-0.005753,-0.002767,-0.007220
4690,2018-08-23,0.002046,-0.003328,-0.079745,0.002436,-0.014451,-0.004837,0.005713,-0.005413,0.000198,...,-0.004192,0.000000,-0.020397,-0.004167,0.002604,-0.011005,-0.020051,-0.007233,-0.009062,-0.007273


In [57]:
historical_stocks=pd.read_csv("industry-data.csv")
historical_stocks["ticker"] = historical_stocks["symbol"]
historical_stocks = historical_stocks.drop(columns=["Unnamed: 0", "symbol"])
historical_stocks

,industry,sector,ticker
0,Mining/Minerals,BASIC INDUSTRIES,20MICRONS
1,Finance & Investments,FINANCE,360ONE
2,Diversified,CONSUMER DURABLES,3MINDIA
3,Finance & Investments,FINANCE,3PLAND
4,IT Consulting & Software,TECHNOLOGY,3IINFOLTD
...,...,...,...
2021,Pharmaceuticals,HEALTH CARE,ZYDUSLIFE
2022,Pharmaceuticals,HEALTH CARE,ZYDUSWELL
2023,IT Enabled Services,TECHNOLOGY,ECLERX
2024,IT Enabled Services,TECHNOLOGY,EMUDHRA


In [58]:
historical_stocks = historical_stocks.loc[historical_stocks.index.repeat(2)].reset_index(drop=True)[["ticker", "sector", "industry"]]

In [59]:
for i in range(len(historical_stocks)):
    if(i%2==0):
        historical_stocks.loc[i, "ticker"] += ".NS"
    else:
        historical_stocks.loc[i, "ticker"] += ".BO"

In [60]:
historical_stocks.to_csv("nse_historical_stocks.csv")

In [61]:
list(historical_stocks.ticker)

['20MICRONS.NS',
 '20MICRONS.BO',
 '360ONE.NS',
 '360ONE.BO',
 '3MINDIA.NS',
 '3MINDIA.BO',
 '3PLAND.NS',
 '3PLAND.BO',
 '3IINFOLTD.NS',
 '3IINFOLTD.BO',
 '5PAISA.NS',
 '5PAISA.BO',
 '63MOONS.NS',
 '63MOONS.BO',
 'A2ZINFRA.NS',
 'A2ZINFRA.BO',
 'AAATECH.NS',
 'AAATECH.BO',
 'ABB.NS',
 'ABB.BO',
 'ABMINTLLTD.NS',
 'ABMINTLLTD.BO',
 'ACC.NS',
 'ACC.BO',
 'ADFFOODS.NS',
 'ADFFOODS.BO',
 'AGI.NS',
 'AGI.BO',
 'AGSTRA.NS',
 'AGSTRA.BO',
 'AIAENG.NS',
 'AIAENG.BO',
 'GOLDTECH.NS',
 'GOLDTECH.BO',
 'AKG.NS',
 'AKG.BO',
 'AMDIND.NS',
 'AMDIND.BO',
 'AMJLAND.NS',
 'AMJLAND.BO',
 'APLAPOLLO.NS',
 'APLAPOLLO.BO',
 'ARSSINFRA.NS',
 'ARSSINFRA.BO',
 'ARTNIRMAN.NS',
 'ARTNIRMAN.BO',
 'ASKAUTOLTD.NS',
 'ASKAUTOLTD.BO',
 'AVG.NS',
 'AVG.BO',
 'AVTNPL.NS',
 'AVTNPL.BO',
 'AXISCADES.NS',
 'AXISCADES.BO',
 'AYMSYNTEX.NS',
 'AYMSYNTEX.BO',
 'AADHARHFC.NS',
 'AADHARHFC.BO',
 'AAKASH.NS',
 'AAKASH.BO',
 'AAREYDRUGS.NS',
 'AAREYDRUGS.BO',
 'AARON.NS',
 'AARON.BO',
 'AARTECH.NS',
 'AARTECH.BO',
 'AARTIDRUGS.N

In [62]:
import os
from tqdm import tqdm

stocks_to_use = set(pd.read_csv("company-datewise.csv").columns)

file_names = sorted([csv_file for csv_file in os.listdir("kaggle-data") if csv_file[:-4] in list(historical_stocks.ticker) and csv_file[-6:-4]=='NS'])

In [63]:
len(file_names)

817

In [64]:

list_dates = []
chosen_file_names = []
for file_name in tqdm(file_names):

    s = pd.read_csv("kaggle-data/"+file_name)
    s['Date'] = pd.to_datetime(s['Date'])

    list_dates.append(set(s['Date']))
    chosen_file_names.append(file_name)

100%|██████████| 817/817 [00:10<00:00, 81.60it/s] 


In [65]:
len(list_dates)

817

In [66]:
min(sorted(set.intersection(*list_dates)))

Timestamp('2018-04-04 00:00:00')

In [67]:
df_returns = pd.DataFrame({"Date": sorted(set.intersection(*list_dates))})

In [68]:
df_returns

,Date
0,2018-04-04
1,2018-04-05
2,2018-04-06
3,2018-04-09
4,2018-04-10
...,...
504,2020-05-26
505,2020-05-27
506,2020-05-28
507,2020-05-29


In [69]:
for file_name in tqdm(chosen_file_names):
    s = pd.read_csv("kaggle-data/"+file_name)
    s['Date'] = pd.to_datetime(s['Date'])

    s[file_name[:-4]] = s['Close']

    df_returns = df_returns.merge(s[[file_name[:-4], 'Date']], left_on="Date", right_on="Date")
    
df_returns

100%|██████████| 817/817 [00:08<00:00, 93.74it/s] 


,Date,20MICRONS.NS,3MINDIA.NS,3PLAND.NS,5PAISA.NS,63MOONS.NS,A2ZINFRA.NS,AARTIDRUGS.NS,AARTIIND.NS,AARVEEDEN.NS,...,VSSL.NS,VSTIND.NS,VSTTILLERS.NS,VTL.NS,WABAG.NS,ZENSARTECH.NS,ZODIACLOTH.NS,ZOTA.NS,ZUARI.NS,ZYDUSWELL.NS
0,2018-04-04,49.299999,20105.000000,13.25,211.054001,101.050003,27.900000,543.799988,577.299988,44.200001,...,148.399994,3096.300049,2707.449951,1263.050049,488.899994,189.119995,169.350006,154.820999,494.549988,1186.800049
1,2018-04-05,49.950001,20638.699219,13.90,209.289993,97.449997,28.750000,563.099976,593.299988,45.150002,...,154.449997,3139.550049,2582.399902,1238.199951,490.399994,188.500000,169.100006,156.429001,496.700012,1208.300049
2,2018-04-06,49.450001,20715.800781,14.00,209.076996,98.250000,29.000000,584.900024,590.825012,46.000000,...,154.899994,3170.600098,2664.949951,1267.699951,494.850006,190.389999,173.550003,156.356995,505.750000,1203.000000
3,2018-04-09,48.950001,20484.500000,15.05,203.938004,98.199997,29.549999,596.349976,590.724976,46.000000,...,155.899994,3107.000000,2703.750000,1259.099976,496.299988,190.699997,175.050003,156.679001,510.000000,1211.900024
4,2018-04-10,48.700001,20663.400391,14.50,202.569000,96.949997,28.799999,582.099976,592.875000,45.900002,...,154.399994,3104.850098,2701.250000,1240.150024,511.100006,190.979996,174.000000,156.429001,504.700012,1202.150024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2020-05-26,25.450001,17520.699219,4.70,147.899994,51.400002,3.750000,883.950012,1009.849976,8.200000,...,43.450001,3135.699951,900.299988,633.049988,90.750000,88.500000,95.949997,139.000000,82.400002,1241.300049
570,2020-05-27,25.600000,17896.900391,4.70,154.350006,52.200001,3.700000,879.700012,986.500000,8.150000,...,44.000000,3073.149902,900.599976,628.750000,89.699997,90.099998,99.400002,139.850006,81.550003,1223.050049
571,2020-05-28,26.100000,18014.300781,4.50,154.850006,52.200001,3.850000,936.849976,983.500000,8.500000,...,44.000000,3084.199951,910.500000,635.150024,91.449997,89.800003,102.849998,138.500000,82.500000,1243.550049
572,2020-05-29,26.150000,18299.199219,4.50,156.699997,54.799999,4.000000,972.250000,976.150024,8.450000,...,44.849998,3114.399902,916.349976,657.500000,91.050003,86.849998,104.800003,138.250000,82.300003,1252.949951


In [70]:
df_returns = df_returns.drop(columns=(df_returns.isna().sum()[df_returns.isna().sum() != 0].index))

In [71]:
df_returns = df_returns.ffill()

In [72]:
df_returns.to_csv("returns_df_528_nse.csv")

In [73]:
df_returns

,Date,20MICRONS.NS,3MINDIA.NS,3PLAND.NS,5PAISA.NS,63MOONS.NS,A2ZINFRA.NS,AARTIDRUGS.NS,AARTIIND.NS,AARVEEDEN.NS,...,VSSL.NS,VSTIND.NS,VSTTILLERS.NS,VTL.NS,WABAG.NS,ZENSARTECH.NS,ZODIACLOTH.NS,ZOTA.NS,ZUARI.NS,ZYDUSWELL.NS
0,2018-04-04,49.299999,20105.000000,13.25,211.054001,101.050003,27.900000,543.799988,577.299988,44.200001,...,148.399994,3096.300049,2707.449951,1263.050049,488.899994,189.119995,169.350006,154.820999,494.549988,1186.800049
1,2018-04-05,49.950001,20638.699219,13.90,209.289993,97.449997,28.750000,563.099976,593.299988,45.150002,...,154.449997,3139.550049,2582.399902,1238.199951,490.399994,188.500000,169.100006,156.429001,496.700012,1208.300049
2,2018-04-06,49.450001,20715.800781,14.00,209.076996,98.250000,29.000000,584.900024,590.825012,46.000000,...,154.899994,3170.600098,2664.949951,1267.699951,494.850006,190.389999,173.550003,156.356995,505.750000,1203.000000
3,2018-04-09,48.950001,20484.500000,15.05,203.938004,98.199997,29.549999,596.349976,590.724976,46.000000,...,155.899994,3107.000000,2703.750000,1259.099976,496.299988,190.699997,175.050003,156.679001,510.000000,1211.900024
4,2018-04-10,48.700001,20663.400391,14.50,202.569000,96.949997,28.799999,582.099976,592.875000,45.900002,...,154.399994,3104.850098,2701.250000,1240.150024,511.100006,190.979996,174.000000,156.429001,504.700012,1202.150024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2020-05-26,25.450001,17520.699219,4.70,147.899994,51.400002,3.750000,883.950012,1009.849976,8.200000,...,43.450001,3135.699951,900.299988,633.049988,90.750000,88.500000,95.949997,139.000000,82.400002,1241.300049
570,2020-05-27,25.600000,17896.900391,4.70,154.350006,52.200001,3.700000,879.700012,986.500000,8.150000,...,44.000000,3073.149902,900.599976,628.750000,89.699997,90.099998,99.400002,139.850006,81.550003,1223.050049
571,2020-05-28,26.100000,18014.300781,4.50,154.850006,52.200001,3.850000,936.849976,983.500000,8.500000,...,44.000000,3084.199951,910.500000,635.150024,91.449997,89.800003,102.849998,138.500000,82.500000,1243.550049
572,2020-05-29,26.150000,18299.199219,4.50,156.699997,54.799999,4.000000,972.250000,976.150024,8.450000,...,44.849998,3114.399902,916.349976,657.500000,91.050003,86.849998,104.800003,138.250000,82.300003,1252.949951


In [74]:
import pandas as pd
import sklearn.preprocessing

returns = pd.read_csv("returns_df_528_nse.csv").drop(columns="Unnamed: 0")
returns[returns.columns[1:]] = sklearn.preprocessing.normalize(returns.drop(columns="Date"))

In [75]:
returns.to_csv("returns_df_528_nse_norm.csv")